QIIME2(설치 Virtual Box)  
https://docs.google.com/presentation/d/1P6udhJDnvC2eurthDtHubcg4e_yDTsbsUNnecMM_gU8/edit?usp=sharing  
https://docs.qiime2.org/2019.1/ (tutorial list)  
하기 내용은 상기 링크를 정리한 것이다.  

# QIIME2 Tutorial

## 설치 환경
* Ubuntu 16.04
* ANACONDA (가상환경, qiime2로 naming한다.) python 3.5.5(버전 3.6에서도 설치 확인)  
  
## QIIME2 tutorial list
* “Moving Pictures” tutorial
* Fecal microbiota transplant (FMT) study: an exercise
* “Atacama soil microbiome” tutorial
* Differential abundance analysis with gneiss
* Importing data
* Exporting data
* Metadata in QIIME 2
* Filtering data
* Training feature classifiers with q2-feature-classifier
* Evaluating and controlling data quality with q2-quality-control
* Predicting sample metadata values with q2-sample-classifier
* Performing longitudinal and paired sample comparisons with q2-longitudinal
* Identifying and filtering chimeric feature sequences with q2-vsearch
* Alternative methods of read-joining in QIIME 2
* Clustering sequences into OTUs using q2-vsearch


## “Moving Pictures” tutorial
https://docs.qiime2.org/2019.1/tutorials/moving-pictures/   
이번 분석은 5개의 시점을 두고 두 사람의 신체에서 추출한 신체 4곳의 미생물에 대해서 분석을 진행할 것이다.  
첫째로 antibiotic(항생물질)을 사용한다. 
  
## sample metadata
> ```$ mkdir Moving_Pictures```  
```$ cd Moving_Pictures```    
```(qiime2) $ wget -O "sample-metadata.tsv" "http://data.qiime2.org/2019.1/tutorials/moving-pictures/sample_metadata.tsv"```

### Obtaining and importing data  
  
> ```(qiime2) $ wget -O "emp-single-end-sequences/barcodes.fastq.gz" "https://data.qiime2.org/2019.1/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz"```  
```(qiime2) $ wget -O "emp-single-end-sequences/sequence.fastq.gz" "https://data.qiime2.org/2019.1/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz"```

  
semantic type QIIME 2는 EMPSingleEndSequence이다. 
EMPSingleEndSequence artifacts는 아직 sequence가 sample에 지정되지 않은  multiplexed한 sequence에 담겨있다.(따라서, sequence.fastq.gz 및 barcodes.fastq.gz 파일을 포함한다. barcodes.fastq.gz에는 sequence.fastq.gz의 각 sequence에 barcode가 들어있다.)
>```(qiime2) $ qiime tools import --type EMPSingleEndSequences --input-path emp-single-end-sequences/ --output-path emp-single-end-sequences.qza```

### Demultiplexing sequences
demultiplex sequence는 각 sample 별로 관련된 barcode sequence를 아는 것이 필요하다. 
이 정보에는 sample metadata file이 담겨있다. 
command를 따라하면 sequences(demux emp-single command는 sequence에  barcode한 Earth Microbiome Project protocol 참조한다.)를 demultiplex할 수 있다.
demultiplexed sequences의 demux.qza를 QIIME2로 생성할 수 있다.  
  
> ```(qiime2) $ qiime demux emp-single \```  
```--i-seqs emp-single-end-sequences.qza \```  
```--m-barcodes-file sample-metadata.tsv \```  
```--m-barcodes-column BarcodeSequence \```  
```--o-per-sample-sequences demux.qza```
  
Demultiplexing 이후 생성된 파일을 이용하여 demultiplexing summary results를 생성한다.

> ```(qiime2) $ qiime demux summarize --i-data demux.qza --o-visualization demux.qzv```

* Visualization 

> ```(qiime2) $ qiime tools view demux.qzv```  
  
or https://view.qiime2.org/ 접속하여 파일을 드래그앤 드랍한다.  

예제) : https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fdemux.qzv 



### Sequence quality control and feature table construction
  
QIIME 2 plugins은 몇가지(DADA2, Deblur, basic quality-score-based filtering) 질좋은 control method 이용할 수 있다.  
Tutorial에서는 DADA2와 Deblur를 사용하며 원하는 것으로 변경하면서 사용해도 무방하다.


#### Option 1: DADA2
DADA2는 Illumina amplicon sequence data를 detecting하고 correcting하는 pipeline이다.  
`q2-dada2` plugin을 통하여 사용이 가능하며, quality control process는 sequencing data에서 식별된 phiX판독 값을 추가적으로 filter한다.  
`*data2 denoise-single` method는 quality filtering에서 사용할 두가지 parameter를 요구한다.  
`--p-trim-left m`의 m에 시작하는 각 sequence 값을 넣고, `--p-trunc-len n`의 n에는 각 시퀀스를 자를 위치에 넣는다.   
위와 같이 pipeline을 통과하면 low quality sequences가 제거된다.



위와 같은 이유 때문에 m, n 값을 찾기 위해 이전단계에서 만들어진 `demux.qzv`파일에서 생성된 qiime demux summarize의 Interactive Quality Plot tab을 확인해야한다.  
`demux.qzv`의 quality plots을 살펴보면 초기 sequence 에서는 quality가 좋은 상황을 볼 수 있으면 sequence가 후반으로 갈수록 quality가 떨어지는 것을 확인 할 수 있다.  
때문에 넉넉히 120 position  drop off할 것이다.  
최대 10분까지 걸리며 모든 과정중에 제일 오래 걸린다.

> ```(qiime2) $ qiime dada2 denoise-single \ ```  
```--i-demultiplexed-seqs demux.qza \```   
```--p-trim-left 0 \```  
```--p-trunc-len 120 \```  
```--o-representative-sequences rep-seqs-data2.qza \```   
```--o-table table-data2.qza \```  
```--o-denoising-stats stats-data2.qza```

#### dada2 Output visualization
> ```(qiime2) $ qiime metadata tabulate \```  
```--m-input-file stats-data2.qza \```  
```--o-visualization stats-dada2.qzv```

dada2로 Tutorial을 지속 진행한다면, 아래와 같이 변환해준다.  
`$ mv rep-seqs-data2.qza rep-seqs.qza`   
`$ mv table-data2.qza table.qza`



#### Option 2 : Deblur

Deblur는 sequence error profile를 이용하여 읽은 erroneous sequence와 true biological sequence를 사용해 high quality sequence variant data를 생성한다.  
  
data 생성시에는 두 단계로 적용된다.
1. quality scores를 적용한 initial(처음의) quality filtering process를 기반으로 한다.  
Deblur는 다음의 [method](https://www.nature.com/articles/nmeth.2276)를 quality filtering approach로 이행한다.
> `(qiime2) $ qiime quality-filter q-score \`  
```--i-demux demux.qza \```  
```--o-filtered-sequences demux-filtered.qza \```  
```--o-filter-stats demux-filter-stats.qza```  
2. ```qiime deblur denoise-16S method``` 를 사용한다.  
다음 method는 ```-p-trim-length n```에서 사용할 parameter(quality filtering을 위한)가 필요하다.  
이때 `n`에는 축소한 sequence가 들어가야한다.  
일반적으로 Deblur를 사용하는 사용자들에게 추천하는 setting은 median quality scroe로 시작하고 drop이 적은 length를 넣는것이다.  
data에서 보듯이 115에서 130의 sequence position range를 제한한다.(QIIME 개발자의 주관적인 관점이므로 축소하거나 늘려도 상관없음)  
추천사항에서 예외적인 상황은 multiple sequencing을 실행하는 상황이다.  
위와 같은 meta-analysis에서 study-specific가 편향 적용되는 것으 피하기 위해 모두 같은 length로 읽는 것이 critical하다.

이미 qiime dada2 denosie-single에서 trim 120 lenght를 사용하였으므로, 120 lenght로 quality plots을 준다.  
> `(qiime2) $ qiime deblur denoise-16S \`  
`--i-demultiplexed-seqs demux-filtered.qza \`  
`--p-trim-length 120 \`  
`--o-representative-sequences rep-seqs-deblur.qza \`  
`--o-table table-deblur.qza \`  
`--p-sample-stats \`  
`--o-stats deblur-stats.qza`  
  
<이름변환>
>`$ mv rep-seqs-deblur.qza rep-seqs2.qza`  
`$ mv table-deblur.qza table2.qza`  

### FeatureTable and FeatureData summaries  
quality filtering단계가 끝난 후, 결과를 표시해야한다.  
아래 command를 따라서 visual summaries를 진행한다.   
Visual summaries는 각 sample과 관련된 많은 히스토그램, 요약 통계를 이용한 많은 정보를 제공할 것이다.   
feature-table tabulate-seqs 명령어는 각 feature Id의 sequense를 제공할것이며,   
또한 반대의 NCBI nt database 쉬운 sequence의 BLAST 도 제공할 것이다.(해석에 대한 수정필요..)  
다음의 visualization은 tutorial 중에서도 매우 유용하게 사용될 것이다.
> `(qiime2) $ qiime feature-table summarize `  
`--i-table table2.qza \`  
`--o-visualization table2.qzv \`  
`--m-sample-metadata-file sample-metadata.tsv`  
  
result link:  
https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Ftable.qzv  

> `(qiime2) $ qiime feature-table tabulate-seqs \`  
`--i-data rep-seqs2.qza \`  
`--o-visualization rep-seqs2.qzv`  

result link:  
https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Frep-seqs.qzv

### Generate a tree for phylogenetic diversity analyses  
QIIME은 Faith’s Phylogenetic Diversity와 weighted 그리고 unweighted UniFrac를 포함한 phylogenetic diversity metric에 대한 몇가지를 지원한다.  
샘플당 feature 수 이외에, 측정기준 feature들에 대한 관계를 형성하는 rooted phylogenetic tree의 metrics가 필요하다.  
정보는 `Phylogeny[Rooted]`에 저장된다.

q2-phylogeny plugin으로 부터 나온 `align-to-tree-mafft-fastree`의 pipeline을 이용하여 plugin을 생성한다.  
첫번째로 pipeline은 QIIME artifact 부터 생성한 multiple sequence로 설정된 mafft program을 사용한다.  
다음은 pipeline filter(or masks)을 이용하여 highly variable 위치를 제거한다.  
이후 position들의 phylogenetic tree 부터 생성된 noise을 일반적으로 고려한다.(수정 필요..)  
> `(qiime2) $ qiime phylogeny align-to-tree-mafft-fasttree \`  
`--i-sequences rep-seqs2.qza \`  
`--o-alignment aligned-rep-seqs.qza \`  
`--o-masked-alignment masked-aligned-rep-seqs.qza \`  
`--o-tree unrooted-tree.qza `



### Alpha and beta diversity analysis
QIIME2는 `q2-diversity plugin`를 이용하여 diversity analysis를 진행할 수 있다.   
plugin은 alpha와 beta diversity metric를 계산을 지원하며 통계적으로 test하는 것도 지원하고 visualization과 상호작용 하는 것도 지원한다.  
처음으로는 `core-metrics-phylogenetic` method를 이용해야한다.  
Alpha와 Beta diversity의 analysis방법의 default는 다음과 같다.  
* Shannon’s diversity index (a quantitative measure of community richness)
* Observed OTUs (a qualitative measure of community richness)
* Faith’s Phylogenetic Diversity (a qualitative measure of community richness that incorporates phylogenetic relationships between the features)
* Evenness (or Pielou’s Evenness; a measure of community evenness)
* Jaccard distance (a qualitative measure of community dissimilarity)
* Bray-Curtis distance (a quantitative measure of community dissimilarity)
* unweighted UniFrac distance (a qualitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)
* weighted UniFrac distance (a quantitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)  
  
중요한 parameter는 --p-sampling-depth 를 이용하는 것이 필요하다.  
각 sampling 짝수로 depth에 적용해야한다.  
왜냐하면 대부분의 diversity metrics는 다른 sample에 다른 sampling depth에 민감하기 때문이다.  
이번 script는 각 제공된 sample count를 매개 변수로 제공된 값으로 임의적으로 subsample한다.  
> `(qiime2) $ qiime diversity core-metrics-phylogenetic \`  
`--i-phylogeny rooted-tree.qza \`  
`--i-table table.qza \`  
`--p-sampling-depth 1109 \`  
`--m-metadata-file sample-metadata.tsv \`  
`--output-dir core-metrics-results`  
  
#### Output visualization
* core-metrics-results/unweighted_unifrac_emperor.qzv :
    * [link](https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted_unifrac_emperor.qzv)
* core-metrics-results/jaccard_emperor.qzv :
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fjaccard_emperor.qzv)
* core-metrics-results/bray_curtis_emperor.qzv : 
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fbray_curtis_emperor.qzv)
* core-metrics-results/weighted_unifrac_emperor.qzv : 
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fweighted_unifrac_emperor.qzv)


Diversity metrics를 계산 후 샘플들을의 microbial composition 시작해야한다.  
sample metadata의 대한 파일은 [링크](https://docs.google.com/spreadsheets/d/1yFL_Mwh7b5LE5vW06KooZFB_cjoEJEXGFwhos6X7jns/edit#gid=0)와 같다.  
* alpha-diversity(faith-pd-group-significance)
> `(qiime2) $ qiime diversity alpha-group-significance \`  
`--i-alpha-diversity core-metrics-results/faith_pd_vector.qza \`  
`--m-metadata-file sample-metadata.tsv \`   
`--o-visualization core-metrics-results/faith-pd-group-significance.qzv`
* alpha-diversity(evenness-pd-group-significance)
> `(qiime2) $ qiime diversity alpha-group-significance \`  
`--i-alpha-diversity core-metrics-results/evenness_vector.qza \`  
`--m-metadata-file sample-metadata.tsv \`  
`--o-visualization core-metrics-results/evenness-group-significance.qzv`  
  
  
* core-metrics-results/faith-pd-group-significance.qzv:
    * [link](https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Ffaith-pd-group-significance.qzv)
* core-metrics-results/evenness-group-significance.qzv:
    * [link](https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fevenness-group-significance.qzv)

다음은 beta-diversity에 대한 코드이다.  
input distance로 사용할 matrix는 unweighted UniFrac distances이며 두개의 샘플코드로 이루어졌다.


* beta-diversity(unweighted-unifrac-body-site-significance)
> `(qiime2) $ qiime diversity beta-group-significance \`  
  `--i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \`  
  `--m-metadata-file sample-metadata.tsv \`  
  `--m-metadata-column BodySite \`  
  `--o-visualization core-metrics-results/unweighted-unifrac-body-site-significance.qzv \`  
  `--p-pairwise`  
* beta-diversity(unweighted-unifrac-subject-group-significance)
> `(qiime2) $ qiime diversity beta-group-significance \`  
  `--i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \`  
  `--m-metadata-file sample-metadata.tsv \`  
  `--m-metadata-column Subject \`  
  `--o-visualization core-metrics-results/unweighted-unifrac-subject-group-significance.qzv \`  
  `--p-pairwise`
* core-metrics-results/unweighted-unifrac-body-site-significance.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted-unifrac-body-site-significance.qzv)
* core-metrics-results/unweighted-unifrac-subject-group-significance.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted-unifrac-subject-group-significance.qzv)  
  
Again, none of the continuous sample metadata that we have for this data set are correlated with sample composition, so we won’t test for those associations here. If you’re interested in performing those tests, you can use the qiime metadata distance-matrix in combination with qiime diversity mantel and qiime diversity bioenv commands.

### Emperor
마지막은 microbial community composition approach에 대한 것이다.  
Emperor tool을 이용하여 PCoA(principal coordinates) plot을 탐색 할 수 있다.  
  
While our core-metrics-phylogenetic command did already generate some Emperor plots, we want to pass an optional parameter, --p-custom-axes, which is very useful for exploring time series data. The PCoA results that were used in core-metrics-phylogeny are also available, making it easy to generate new visualizations with Emperor. We will generate Emperor plots for unweighted UniFrac and Bray-Curtis so that the resulting plot will contain axes for principal coordinate 1, principal coordinate 2, and days since the experiment start. We will use that last axis to explore how these samples changed over time.  
> `(qiime2) $ qiime emperor plot \`  
  `--i-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza \`  
  `--m-metadata-file sample-metadata.tsv \`  
  `--p-custom-axes DaysSinceExperimentStart \`  
  `--o-visualization core-metrics-results/unweighted-unifrac-emperor-DaysSinceExperimentStart.qzv`  
  
  
> `(qiime2) qiime emperor plot \`  
  `--i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza \`  
  `--m-metadata-file sample-metadata.tsv \`  
  `--p-custom-axes DaysSinceExperimentStart \`  
  `--o-visualization core-metrics-results/bray-curtis-emperor-DaysSinceExperimentStart.qzv`  
  
* core-metrics-results/unweighted-unifrac-emperor-DaysSinceExperimentStart.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted-unifrac-emperor-DaysSinceExperimentStart.qzv)
* core-metrics-results/bray-curtis-emperor-DaysSinceExperimentStart.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fbray-curtis-emperor-DaysSinceExperimentStart.qzv)
    
### Alpha rarefaction plotting
QIIME의 alpha-diversity를 이용하여 rarefaction visualizer를 해보도록 한다.
> `(qiime2) $ qiime diversity alpha-rarefaction \`  
`--i-table table.qza \`  
`--i-phylogeny rooted-tree.qza \`  
`--p-max-depth 4000 \`  
`--m-metadata-file sample-metadata.tsv \`  
`--o-visualization alpha-rarefaction.qzv`  
  
* alpha-rarefaction.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Falpha-rarefaction.qzv)

### Taxonomic analysis
다음은 taxonomic composition 에 대한 섹션이다.  
처음에는 QIIME에 taxonomy sequences를 지정하는 것이다.  
이때 pre-trained Naive Bayes classifier와 q2-feature-classifier plugin을 사용할 것이다.  
classifier 학습은 Greengenes 13_8 99% OTUs로 할 것이다.  
> `$ wget \`  
  `-O "gg-13-8-99-515-806-nb-classifier.qza" \`  
  `"https://data.qiime2.org/2019.1/common/gg-13-8-99-515-806-nb-classifier.qza"`  
`$ pip install scikit-learn==0.20.2`

> `(qiime2) $ qiime feature-classifier classify-sklearn \`  
  `--i-classifier gg-13-8-99-515-806-nb-classifier.qza \`  
  `--i-reads rep-seqs.qza \`  
  `--o-classification taxonomy.qza`  
  
> `(qiime2) $ qiime metadata tabulate \`  
  `--m-input-file taxonomy.qza \`  
  `--o-visualization taxonomy.qzv`
  
* taxonomy.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Ftaxonomy.qzv)  
    
Next, we can view the taxonomic composition of our samples with interactive bar plots. Generate those plots with the following command and then open the visualization.  
> `(qiime2) $ qiime taxa barplot \`  
  `--i-table table.qza \`  
  `--i-taxonomy taxonomy.qza \`  
  `--m-metadata-file sample-metadata.tsv \`  
  `--o-visualization taxa-bar-plots.qzv`  
  
* taxa-bar-plots.qzv:
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Ftaxa-bar-plots.qzv)  

----

## Fecal microbiota tansplant(FMT) study: an exercise

Tutorial을 위하여 data는 [Fecal Microbiome Transplant study](https://microbiomejournal.biomedcentral.com/articles/10.1186/s40168-016-0225-7)에서 받아왔다.  
FMT는 autism(자폐증)과 gastrointestinal disorders(위장장애)를 가진 18세 이하 아이들의 변을 Autism Diagnostic Interview-Revised(ADI-R: 위장 증상 평가 척도)및,   
Gastrointestinal Symptom Rating Scale(GSRS)로 microbiota의 움직임과 증상으로 위장 증상을 완화하기 위해 측정한 데이터이다.  
  
The Parent Global Impressions-III(PGI-III)과 Childhood Autism Rating Scale(CARS)를 포함한 위장 심각도에 대한 몇가지 측정 기준으로 위장 미생물의 18주 동안 변화를 통해 심각도를 점수화한다.  
microbiome은 배설물 면봉 sample(화장실에서 사용된 휴지로 부터 수집됨)과 적은 횟수의 배설물 sample을 한주 동안 모은 것으로 추적되었다.  
  
전체 연구에서, 실험의 안정성을 위해 18명 환자는 치료를 받았고 20명의 환자는 대조군으로 남았다.  
대조군은 치료 받지 않았으나 gut microbiom의 변화를 확인하기위해  주기적을로 monitor하였다.  
연구에서 치료 하는동안에는 이식된 The fecal material sequenced 분석되었다.  
  
The tutorial dataset은 data의 subsample로 이번 학습에서 생성되었다.  
각 기 다른 5개의 치료 방법으로 치료된 5명의 데이터가 포함되어 있다.  
6, 16은 각 사람의 stool과 fecal swab가 포함된 sample이다.  
이 sample은 FMT치료 이전과 이후의 sample이다.  
5개 sample은 fecal에 transplant한 측정값을 포함하고 있다.  
  
Moving Pictures tutorial과 같이 DADA2를 사용하여 initial quality control을 진행하였고 FeatureTable과 FeatureData object를 생성하였다.   
그러나 DADA2 denoising을 진행할때 single sequencing만 진행하였다.  
때문에 각 sequencing을 실행할때 기본적으로 다음의 결과를 병합해야한다.  
  
### Obtain data file
> `(qiime2) $ mkdir fmt-tutorial`  
`(qiime2) $ cd fmt-tutorial/`  
`(qiime2) $ wget \`  
  `-O "sample-metadata.tsv" \`  
  `"https://data.qiime2.org/2019.1/tutorials/fmt/sample_metadata.tsv"`  
  
다음으로 필요한 것은 demultiplexed analysis를 진행할 sequences데이터를 받아야 한다.  
두개의 demultiplexed sequences set을 받아야하며, 각 sequence를 동일하게 실행해야 한다.  

이번 tutorial에서는 complete sequence data의 small subsample이용하여 빠르게 실행 할 수 있다.  
현재 진행하는 test는 10% subsample data를 이용한다.  
  
> `(qiime2) $ wget -O "fmt-tutorial-demux-1.qza" "https://data.qiime2.org/2019.1/tutorials/fmt/fmt-tutorial-demux-1-10p.qza"`  
`(qiime2) $ wget -O "fmt-tutorial-demux-2.qza" "https://data.qiime2.org/2019.1/tutorials/fmt/fmt-tutorial-demux-2-10p.qza"`

### Sequence Quality control
quality control을 위하여 demultiplexed sequences를 DADA2에 사용할 것이다.  
**그러나**   
demultiplexed sequnces에 개별적으로 denoise-single명령어를 실행한다.   
> `(qiime2) $ qiime demux summarize \`  
`--i-data fmt-tutorial-demux-1.qza \`  
`--o-visualization demux-summary-1.qzv`  
  
* demux-summary-1.qzv:  
[link](https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Ffmt%2Fdemux-summary-1.qzv)  

> `(qiime2) $ qiime demux summarize \`  
`--i-data fmt-tutorial-demux-2.qza \`  
`--o-visualization demux-summary-2.qzv`  
  
* demux-summary-2.qzv:  
[link](https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Ffmt%2Fdemux-summary-2.qzv)  
  
Here the quality seems relatively low in the first few bases, and then seems to stay relatively high through the end of the reads.   
We’ll therefore trim the first 13 bases from each sequence and truncate the sequences at 150 bases.  
Since the reads are 151 bases long, this results in very little truncation of the sequences.
  
> `(qiime2) $ qiime dada2 denoise-single \`  
`--p-trim-left 13 \`  
`--p-trunc-len 150 \`  
`--i-demultiplexed-seqs fmt-tutorial-demux-1.qza \`  
`--o-representative-sequences rep-seqs-1.qza \`  
`--o-table table-1.qza \`  
`--o-denoising-stats stats-1.qza`  
  
> `(qiime2) $ qiime dada2 denoise-single \`  
`--p-trim-left 13 \`  
`--p-trunc-len 150 \`  
`--i-demultiplexed-seqs fmt-tutorial-demux-2.qza \`  
`--o-representative-sequences rep-seqs-2.qza \`  
`--o-table table-2.qza \`  
`--o-denoising-stats stats-2.qza`  

### Viewing denoising stats
기본적인 denoising process 통계이다.
> `(qiime2) $ qiime metadata tabulate \`  
`--m-input-file stats-1.qza \`  
`--o-visualization denoising-stats-1.qzv`  
  
* denoising-stats-1.qzv:  
[link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Ffmt%2Fdenoising-stats-1.qzv)

> `(qiime2) $ qiime metadata tabulate \`  
`--m-input-file stats-2.qza \`  
`--o-visualization denoising-stats-2.qzv`  
  
* denoising-stats-2.qzv:   
[link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Ffmt%2Fdenoising-stats-2.qzv)




### Merging denoised data  
Merging denoised data step은 각 sequencing 을 실행하는 마지막 단계이다.  
denosined된 두개의 sequence를 합치는 단계로서 table1과 table2를 합치면된다.  
(in this case, the feature id is the md5 hash of the sequence defining the feature).  
  
table merge
> `(qiime2) $ qiime feature-table merge \`  
`--i-tables table-1.qza \`  
`--i-tables table-2.qza \`  
`--o-merged-table table.qza `
  
representative sequence merge  
> `(qiime2) $ qiime feature-table merge-seqs \`  
`--i-data rep-seqs-1.qza \`  
`--i-data rep-seqs-2.qza \`  
`--o-merged-data rep-seqs.qza`  

### Diversity analysis
FeatureTable, FeatureData객체가 생성되었으므로 metadata로 구성된 sample을 분석 할 준비가 되었다.
`the moving pictures`에서 사용된 코드를 이용한다.
다음에서 나오는 질문들은 longitudinal 에 대한 분석이다. longitudinal은 q2-longituinal tutorial에서 확인할 수 있다.
각 질문에 대한 결과를 수집하는게 목적이다.
  
https://docs.qiime2.org/2019.1/tutorials/fmt/ 질문에 대한 링크 (추후에 진행)

----

## "Atacama soil microbiome" tutorial  
이번 튜토리얼은 두개의 목적이 있다.  
첫번째 목적은 initial processing의 paired-end를 읽는것을 분석하는 것이고,  
동일한 시점 까지 single-end 설명의 읽고 분석을 하여 설명하는 것이다.  
이번 단계에서는 importing, demultiplexing, and denoising step그리고 feature table과 feature와 관련된 결과도 포함되어있다.  
  
The Atacama Desert는 지구상에서 가장 건조한 지역이며, per decade 동안 비가 1millimeter 이하로 비가 내린다.  
그럼에도 불구하고 극단적으로 건조한 상황의 흙속에서 microbiome이 살고 있다.  
이번 연구에서 profiled된 soil microbiome은 높이와 습도가 양의 상관관계에 있는 두 동서 횡단인 Baquedano와 Yungay를 따른다.  
횡단을 따라 각 지역에서 토양샘플을 추출하였고 각기 다른 구덩이의 3가지 깊이에서 수집하였다.  
> `(qiime2) $ mkdir qiime2-atacama-tutorial`  
`(qiime2) $ cd qiime2-atacama-tutorial/`  
`(qiime2) $ wget -O "sample-metadata.tsv" "https://data.qiime2.org/2019.1/tutorials/atacama-soils/sample_metadata.tsv"`  


### 10% subsample data
다음은 multiplexed를 읽는 것이다. forward, reverse, 와 barcode fastq.gz파일을 다운로드해야 한다.
> `$ mkdir emp-paired-end-sequences`  
  
* forword  
  
> `(qiime2) $ wget -O "emp-paried-end-sequences/forward.fastq.gz" \`  
`"https://data.qiime2.org/2019.1/tutorials/atacama-soils/10p/forward.fastq.gz"`
  
* reverse  
  
> `(qiime2) $ wget -O "emp-paried-end-sequences/reverse.fastq.gz" \`  
`"https://data.qiime2.org/2019.1/tutorials/atacama-soils/10p/reverse.fastq.gz"`  
  
* barcodes  
  
> `(qiime2) $ wget -O "emp-paried-end-sequences/barcodes.fastq.gz" \`  
`"https://data.qiime2.org/2019.1/tutorials/atacama-soils/10p/barcodes.fastq.gz"`  
  
### Paired-end read analysis commands

data를 분석하려면 이전에 다운받았던 3가지 `EMPPairedEndSequences` 파일의 유형을 바꿔줘야한다.  
* import  
  
> `(qiime2) $ qiime tools import \`  
`--type EMPPairedEndSequences \`  
`--input-path emp-paried-end-sequences \`  
`--output-path emp-paired-end-sequences.qza`  



### Read demultiplex sequence  
다음은 demultiplex sequence를 읽는 것이다.  
sequence를 읽기 위해서는 metadata file sample을 요구한다.  
그리고 해당 파일에서 per-sample barcode가 포함된 column을 지정해야한다.  
 In this data set, the barcode reads are the reverse complement of those included in the sample metadata file,  
 so we additionally include the --p-rev-comp-mapping-barcodes parameter.  
 #### Barcode mapping
> `(qiime2) $ qiime demux emp-paired \`  
`--m-barcodes-file sample-metadata.tsv \`  
`--m-barcodes-column BarcodeSequence \`  
`--i-seqs emp-paired-end-sequences.qza \`  
`--o-per-sample-sequences demux.qza \`  
`--p-rev-comp-mapping-barcodes`


#### Demultiplex summarize
> `(qiime2) $ qiime demux summarize \`  
`--i-data demux.qza \`  
`--o-visualization demux.qzv`  
  
* demux.qzv:  
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fatacama-soils%2Fdemux.qzv)

### Quality sequence
demultiplexing 이후,  무작위로 선발된 sample을 기반으로 sequence quality를 살펴보고 denoise를 진행해야 한다.  
quality plot을 볼때, “moving pictures tutorial”과는 다르게 두개의 plots의 sample을 가지고 있는 것을 주의해야한다.  
plot left에는 forward로 읽은 quality score를 표시하고, plot right에는 reverse로 읽은 quality score를 표시한다.  
우리는 plot을 사용하여 trimming parameters를 결정하고 dada2 denoise-paired를 이용하여 denoise를 진행한다.   
이번 예제에서는 150의 forward와 reverse로 읽는다.  
Since we need the reads to be long enough to overlap   
paired의 처음과 끝이 연결될때 충분히 길어야 하기 때문에 13개의 forward와 reverse로 trimmed된다,  
그러나 판독 길이가 너무 작아지지 않도록 sequence의 끝 부분에 trimming을 적용하지 않는다.  
> `(qiime2) $ qiime dada2 denoise-paired \`  
`--i-demultiplexed-seqs demux.qza \`  
`--p-trim-left-f 13 \`  
`--p-trim-left-r 13 \`  
`--p-trunc-len-f 150 \`  
`--p-trunc-len-r 150 \`  
`--o-table table.qza \`  
`--o-representative-sequences rep-seqs.qza \`  
`--o-denoising-stats denoising-stats.qza`  


#### Feature table, sequences
이번 단계는 feature table과 그에 따른 feature sequences가 포함된 출력물을 얻는다.  
  
* table
  
> `(qiime2) $ qiime feature-table summarize \`  
`--i-table table.qza \`   
`--o-visualization table.qzv \`  
`--m-sample-metadata-file sample-metadata.tsv`  
  
* visualization  
  
> `(qiime2) $ qiime feature-table tabulate-seqs \`  
`--i-data rep-seqs.qza \`  
`--o-visualization rep-seqs.qzv`  

### Denoising visualize
> `(qiime2) $ qiime metadata tabulate \`  
`--m-input-file denoising-stats.qza \`  
`--o-visualization denoising-stats.qzv`  
  
* denoising-stats.qzv: 
    * [list](https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fatacama-soils%2Fdenoising-stats.qzv) 
  
----

## Differential abundance analysis with gneiss  
https://docs.qiime2.org/2018.11/tutorials/gneiss/  
  
이번 tutorial은 Differential abundance analysis를 어떻게 분석하는 에 대한 tutorial이다.   
이때 분석데이터는 gneiss이다.  
  
여기서 진행할 것은 어떻게 일관적인 방식으로 서로 다른 abundant taxa를 식별하는지이다.  
* Compositionality는 비율을 나타낸다.  
* Abundances는 sequencing depth를 설명하기 위해 비율로 설명한다.(Abundances값은 상대값이다.)  
  
미생물이 변하고 있는지 정확히 추론하는 것은 어렵다.   
이유는 single microbe의 변화로 비율값에 변화가 생기면 이전 microbes 비율도 변화하기 때문이다.  
다음 그림을 보면 확인이 가능하다.  
![](https://docs.qiime2.org/2018.11/_images/gneiss-bars.jpg)<br>
<br>

왼쪽(plot)에는 10종의 대한 true abundance가 있고 첫 번째 종은 time point 1에서 time point 2로 변화하는 동안 두배 증가한다.  
오른쪽(plot)에서 비율로 normalize하면, 모든 species는 두개의 time point에서 변화는 하는 것이 나타난다.  
proportions만 본다면, 정확한 문제에 대해서 깨닫지 못할 것이고, proportions로 기반하여 변하게 되는 것을 정확하게 결정하기 힘들다.  
  
differentially abundant species를 확인하는 것에 대한 문제를 정확하게 해결 할 수 없지만,  
느슨한 방식으로 microbes의 partition가 변화하고 있는지 대해 알아볼 수 있다.  
이전 그래프에서 first species와 second species를 비율로 계산했다면, 비율은  
* time point 1은 1:1이 될 것이고, 
* time point 2는 2:1이 될 것이다.
이 방법은 balance를 해결하려는 질문 유형이다.  
  
individual taxa에 포커스하기 보다는 오히려, taxa(or group of taxa)의 비율에 초점을 맞춘다.  
그 이유는 true abundances와 관찰된 species의 관찰 비율의 비율을 고려하기 때문이다.  
다음은 시각화를 위해 log 변환을 한다.  
multiple species는 tree로 표현이 가능하다.  
  
![](https://docs.qiime2.org/2018.11/_images/gneiss-balances.jpg)  
  
왼쪽을 tree를 보면, 각 tips는 taxon군에 해당하고 그 아래에는 첫번째 sample의 taxon 비율이 있다.  
internal nodes는 두 taxa아래에 log 비율로 정의된다.  
오른쪽에도 같은 모양의 tree가 존재하고 그 아래에는 각기 다른 sample taxa의 비율이다.   
balance containing를 보면 오직 보라색 taxa abundances만 바뀐다.  
b3같은 경우에는 빨간색 taxa 의 비율만 고려하므로 변경되지 않는다.  
  
balance 대신 proportions를 보면 일부 주어진 balance 내부의 taxa만 초점을 맞출 수 있는 variance를 제한 함으로써 제거가 가능하다.  
여기서 가장 중요한 질문은 어떻게 해야 balance tree를 구성하고 interesting differentially abundant한 taxa 분류를 식별할 수 있는가? 이다.  
gneiss는 3가지 main 방법이 있다. 
* Correlation clustering
    * 만약 서로 연관된 organisms끼리 cluster할 수 있는 정보가 없다면, 얼마나 자주 함께 발생하는지에 따라 유기체를 그룹화 할 수 있다.
* Gradient clustering
    * metadata category는 cluster taxa를 similar sample type을 이용한다.
    * 예를 들어, 만약 pH가 중요한지 평가하기를 원한다면, pH에 따라 분류하고 하고 pH가 변할때 low-pH organisms과 high-ph organisms 비율이 변하는 지를 관찰할 수 있다.
* Phylogenetic analysis
    * phylogenetic analysis는 phylogenetic tree에 input값을 ilr-phylogenetic으로 사용한다.
    * tree가 생기면 다음의 equation을 이용하여 balance를 계산할 수 있다.  
  
### Equation
$$
b_i = \sqrt{\frac{rs}{r+s}} log\frac{g(x_r)}{g(x_s)}
$$  
i는 tree안에 내부 노드의 $i^{th}$(i 번째)를 나타낸다.  
$g(x)$는 x set의 기하평균을 나타낸다.  
$x_r$은 분자의 balance안의 taxa abundances의 집합을 나타낸다.  
$x_s$는 분모의 balance안의 taxa abundances의 집합을 나타낸다.  
$r$과 $s$는  $x_r$, $x_s$ 내의 taxa의 수를 나타낸다.  
balances가 계산된후에는 ANOVA 및 선형 회귀 같은 표준 통계 절차를 수행할 수 있다.  


### Creating balances
Chronic Fatigue Syndrome dataset(만성 피로 증후군 데이터)을 사용한다.  
dataset에는 총 87명의 데이터가 존재하고 환자 48명과 정상인 39명의 사람이 있다.  
  
### make env
> `(qiime2) $ mkdir chronic-fatigue-syndrome`  
`(qiime2) $ cd chronic-fatigue-syndrome/`


### Obtain Data  
* table
> `(qiime2) $ wget -O "table.qza" "https://data.qiime2.org/2019.1/tutorials/gneiss/table.qza"`
  
* taxa
> `(qiime2) $ wget -O "taxa.qza" "https://data.qiime2.org/2019.1/tutorials/gneiss/taxa.qza"`


먼저, balance를 구성하고자 하는 microbes의 partition을 정의할 것이다.  
다시말하면, microbes(balances)의 partition을 정의하는 tree(i.e. hierarchy)의 구성하고자하는 여러가지 방법이 있다.  
이때 사용하는 것이 correlation-clustering과 gradient-clustering이 있다.   
  
주의 할 것은 differential abundance techniques는 log ratio 변환을 활용할 것이다.
logarithm(log)에는 0을 취할 수 없기 때문에 clustering method에는 default pseudocount(가짜 계수) parameter를 포함한다.
위와 같이 table에 존재하는 모든 0대신에 parameter를 지정하면, 1로 바뀌므로 log를 적용할 수 있게된다.
  
### Option 1: Correlation-clustering
이 옵션은 기본 옵션이다.  
Principal Balance를 위해  unsupervised clustering을 이용하여을 계층적 군집합(hierarchical clustering)에 사용한다.  
  
$$
d(x, y) \; = \; V[ln \frac{x}{y}]
$$
  
x, y는 모든 표본에서 두 microbes의 비율을 나타낸다.  
만약에 두개의 microbes가 관련성이 높으면 0에 가까워진다.  
이후 hierarchical cluster는 distance metric를 이용하여 관련된 microbes끼리 그룹 clustering을 진행한다.  
> `(qiime2) $ qiime gneiss correlation-clustering --i-table table.qza --o-clustering hierarchy.qza`


### Option 2 : Gradient-clustering
correlation-clustering의 대안은 numeric metadata 기반의 tree를 만드는 것이다.  
Gradient-clustering을 이용하면 비슷한 거리의 metadata category의 taxa 그룹이 발생한다.  
> `(qiime2) $ qiime gneiss gradient-clustering \`  
`--i-table table.qza \`  
`--m-gradient-file sample-metadata.tsv \`  
`--m-gradient-column Age \`  
`--o-clustering gradient-hierarchy.qza`  
  
이때 중요한 문제는 overfitting이다.  
gradient-clustering을 이용하면, 구성상의 차이점을 가장 잘 표현할 수 있는 false positive를 얻을 수 있는 tree를 생성할 수 있다.  
gradient-clustering 주의해서 사용해야한다.  
  
### Building linear models using balances  
이제 partition을 정의한 tree가 생겼으니 isometric log ratio(ILR) transform이 가능하다.  
ILR transform은 그룹간의 tree안의 node의 log ratio을 계산한다.  
> `(qiime2) $ qiime gneiss ilr-hierarchical \`  
`--i-table table.qza \`  
`--i-tree hierarchy.qza \`  
`--o-balances balances.qza`




이제 tree에 있는 각 node의 log ratio를 가지고 linear regression 할 수 있다.  
linear regression은 multivariate response linear regression을 사용할 수 있으며,  
각 balance separately에 대한 linear regression을 요약(boils down)하여 실행한다.  
environmental variables와 microbial abundances를 연관 시키려는 시도할수 있다.  
  
model을 실행하면 standard univariate regression에 비해 여러가지 장점이 있다.  
overfitting과 관련있는 많은 issues에 대해 피하고,  
environmental parameters를 기본으로 하는 community-wide perturbations에 대한 perspective(관점)를 얻을 수 있다.  
  
microbial abundances(농도)에 직접적으로 balances를 mapping할 수 있기 때문에 balances에 대한 multivariate regression directly 수행할 수 있다.  
모델은 다음과 같이 표현된다.  

$$
\overrightarrow{y}\;=\;\overrightarrow{\beta_0}\;+\;\overrightarrow{\beta_{subject}}\;\overrightarrow{X_{subject}}\;+\;\overrightarrow{\beta_{sex}}\;\overrightarrow{X_{sex}}\;+\;\overrightarrow{\beta_{age}}\;\overrightarrow{X_{age}}\;+\; \overrightarrow{\beta_{sCD14ugml}}\;\overrightarrow{X_{sCD14ugml}}\;+\;\overrightarrow{\beta_{LBPugml}}\;\overrightarrow{X_{LBPugml}} 
$$

Where y(vector) represents the matrix of balances to be predicted, $β_i$(vector) represents a vector of coefficients for covariate i and $X_i$(vector)represents the measures for covariate i.  
여기서 y벡터는 예측될 balance의 matrix이며,  
$\beta_i$(vector)는 공변량(covariate) i에 대한 계수(coefficients)의 벡터를 나타내고  
$X_i$(vector)는 공변량 i에 대한 계수를 나타낸다.  
ANOVA는 특별한 케이스 임을 기억하고 있어야하며, ANOVA로 해결할수 있는 모든 문제는 선형회귀로 재구성 할 수 있다.  
> `(qiime2) $ qiime gneiss ols-regression \`  
`--p-formula "Subject+Sex+Age+BMI+sCD14ugml+LBPugml+LPSpgml" \`  
`--i-table balances.qza \`  
`--i-tree hierarchy.qza \`  
`--m-metadata-file sample-metadata.tsv \`  
`--o-visualization regression_summary.qzv`  
  
* regression_summary.qzv
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fgneiss%2Fregression_summary.qzv)

이후 내용은 추후에 진행

### Dendrogram, Heatmap 
  
> `(qiime2) $ qiime gneiss dendrogram-heatmap \`  
`--i-table table.qza \`  
`--i-tree hierarchy.qza \`   
`--m-metadata-file sample-metadata.tsv \`  
`--m-metadata-column Subject \`  
`--p-color-map seismic \`  
`--o-visualization heatmap.qzv \`  
  
* heatmap.qzv:  
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fgneiss%2Fheatmap.qzv)
  
### Balance Taxonomy
  
> `(qiime2) $ qiime gneiss balance-taxonomy \`  
`--i-table table.qza \`  
`--i-tree hierarchy.qza \`  
`--i-taxonomy taxa.qza \`  
`--p-taxa-level 2 \`  
`--p-balance-name 'y0' \`  
`--m-metadata-file sample-metadata.tsv \`  
`--m-metadata-column Subject \`  
`--o-visualization y0_taxa_summary.qzv`  
  
* y0_taxa_summary.qzv:  
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fgneiss%2Fy0_taxa_summary.qzv)


----

## Importing data
https://docs.qiime2.org/2019.1/tutorials/importing/  
  
QIIME2를 사용하기 위해서 input data를 반드시 QIIME2 artifact에 저장해야한다(`.qza` 파일이어야 한다.)  
`.qza`파일은 출처가 자동으로 추적되고 분산이 가능하게 되기 때문이다.  
또한 validation의 의미와 변환도 자동 추적된다.([link](https://docs.qiime2.org/2019.1/concepts/)여기서 concept를 살펴볼 수 있다.)  
이번 튜토리얼은 바이러스 데이터 포맷을 QIIME2에 넣는 것을 진행해본다.  
  
Importing은 보통 initial data에 발생하지만 importing모든 단계에서 수행이 가능하다.  
예를 들어 collaborator provides가 `.biom` format의 feature table을 제공하는 경우 이것을 QIIME 2 artifact에 import하여 “downsteam” statistical analysis를 할 수 있다.  
  
Importing은 QIIME2 interface를 이용하여 완료할 수 있다.  
이 tutorial에서는 2 command-line interface(q2cli)를 이용하여 데이터를 import하는 것에 집중한다.  
아래와 같은 서로 다른 타입의 FASTQ data를 import한다.  
1. FASTQ data with the EMP Protocol format  
2. FASTQ data with the Casava 1.8 demultiplexed format  
3. Any other kind of FASTQ data  





### “EMP protocol” multiplexed single-end fastq   
Format description  
Single-end “Earth Microbiome Project (EMP) protocol” formatted reads should have two fastq.gz files total:  
  
one fastq.gz file that contains the single-end reads,  
and another that contains the associated barcode reads In this format, sequence data is still multiplexed (i.e. you have only one fastq.gz file containing raw data for all of your samples).  
  
The order of the records in the two fastq.gz files defines the association between a sequence read and its barcode read (i.e. the first barcode read corresponds to the first sequence read, the second barcode to the second read, and so on).  

* Obtain data  
  
> `(qiime2) $ mkdir emp-single-end-sequences`   
  
> `(qiime2) $ wget -O "emp-single-end-sequences/barcodes.fastq.gz" \`   
`"https://data.qiime2.org/2019.1/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz"`   
  
> `(qiime2) $ wget -O "emp-single-end-sequences/sequences.fastq.gz" \`
`"https://data.qiime2.org/2019.1/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz"`


### Importing data 
> `(qiime2) $ qiime tools import \`  
`--type EMPSingleEndSequences \`  
`--input-path emp-single-end-sequences \`  
`--output-path emp-single-end-sequences.qza`

### “EMP protocol” multiplex paired-end fastq
Paired-end “Earth Microbiome Project (EMP) protocol” formatted reads should have three fastq.gz files total:  
  
one fastq.gz file that contains the forward sequence reads,  
one fastq.gz file that contains the reverse sequence reads,  
and a third that contains the associated barcode reads  
In this format, sequence data is still multiplexed (i.e. you have only one forward and one reverse fastq.gz file containing raw data for all of your samples).  
  
* Obtain data  
  
> `(qiime2) $ mkdir emp-paired-end-sequences`  
  
> `(qiime2) $ wget -O \`  
`"emp-paired-end-sequences/forward.fastq.gz" \`  
`"https://data.qiime2.org/2019.1/tutorials/atacama-soils/1p/forward.fastq.gz"`
  
> `(qiime2)$ wget -O \`  
`"emp-paired-end-sequences/reverse.fastq.gz" \`  
`"https://data.qiime2.org/2019.1/tutorials/atacama-soils/1p/reverse.fastq.gz"`
  
> `(qiime2)$ wget -O \`  
`"emp-paired-end-sequences/barcodes.fastq.gz" \`  
`"https://data.qiime2.org/2019.1/tutorials/atacama-soils/1p/barcodes.fastq.gz"`





* Importing data

> `(qiime2) $ qiime tools import \`  
`--type EMPPairedEndSequences \`  
`--input-path emp-paired-end-sequences \`  
`--output-path emp-paired-end-sequences.qza`

### Casava 1.8 single-end demultiplexed fastq
Format description  
In the Casava 1.8 demultiplexed (single-end) format, there is one fastq.gz file for each sample in the study which contains the single-end reads for that sample. The file name includes the sample identifier and should look like L2S357_15_L001_R1_001.fastq.gz. The underscore-separated fields in this file name are:  
  
1. the sample identifier,  
2. the barcode sequence or a barcode identifier,  
3. the lane number,  
4. the direction of the read (i.e. only R1, because these are single-end reads), and  
5. the set number.  
  

* Obtaining example data   
> `(qiime2) $  wget \`  
  `-O "casava-18-single-end-demultiplexed.zip" \`  
  `"https://data.qiime2.org/2019.1/tutorials/importing/casava-18-single-end-demultiplexed.zip"`  
  
* unzip data
> `(qiime2) $ unzip -q casava-18-single-end-demultiplexed.zip`  
  
* Importing data
> `(qiime2) $ qiime tools import \`  
  `--type 'SampleData[SequencesWithQuality]' \`  
  `--input-path casava-18-single-end-demultiplexed \`  
  `--input-format CasavaOneEightSingleLanePerSampleDirFmt \`  
  `--output-path demux-single-end.qza`  
  
* demux-single-end.qza:  
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fimporting%2Fdemux-single-end.qza)

### Casava 1.8 paired-end demultiplexed fastq
**Format description**
  
In Casava 1.8 demultiplexed (paired-end) format, there are two fastq.gz files for each sample in the study, each containing the forward or reverse reads for that sample. The file name includes the sample identifier. The forward and reverse read file names for a single sample might look like L2S357_15_L001_R1_001.fastq.gz and L2S357_15_L001_R2_001.fastq.gz, respectively. The underscore-separated fields in this file name are:

1. the sample identifier,
2. the barcode sequence or a barcode identifier,
3. the lane number,
4. the direction of the read (i.e. R1 or R2), and
5. the set number.
  

* Obtaining example data
> `(qiime2) $ wget -O "casava-18-paired-end-demultiplexed.zip" \`   
`"https://data.qiime2.org/2019.1/tutorials/importing/casava-18-paired-end-demultiplexed.zip"`  
  
* unzip
> `(qiime2) $ unzip -q casava-18-paired-end-demultiplexed.zip`  
  
* Importing data
> `(qiime2) $ qiime tools import \`  
  `--type 'SampleData[PairedEndSequencesWithQuality]' \`  
  `--input-path casava-18-paired-end-demultiplexed \`  
  `--input-format CasavaOneEightSingleLanePerSampleDirFmt \`  
  `--output-path demux-paired-end.qza`

### “Fastq manifest” formats

If you don’t have either EMP or Casava format, you need to import your data into QIIME 2 manually by first creating a “manifest file” and then using the qiime tools import command with different specifications than in the EMP or Casava import commands.  
  
Format description
First, you’ll create a text file called a “manifest file”, which maps sample identifiers to fastq.gz or fastq absolute filepaths that contain sequence and quality data for the sample (i.e. these are FASTQ files). The manifest file also indicates the direction of the reads in each fastq.gz or fastq file. The manifest file will generally be created by you, and it is designed to be a simple format that doesn’t put restrictions on the naming of the demultiplexed fastq.gz / fastq files, since there is no broadly used naming convention for these files. You can call the manifest file whatever you want.  
  
The manifest file is a comma-separated (i.e., .csv) text file. The first field on each line is the sample identifier that should be used by QIIME, the second field is the absolute filepath, and the third field is the read direction. Lines beginning with # and blank lines are ignored. The first line in the file that does not begin with a # and is not blank must be the header line: sample-id,absolute-filepath,direction. With the exception of the header line, the order of lines in this file is not important.  
  
For single-end reads, there must be exactly one line per sample ID corresponding to the forward reads (or reverse, if you’re processing these separately as single-end reads). For paired-end reads there must be exactly two lines per sample id, corresponding to the forward and the reverse reads. The direction field on each line can only contain the text forward or reverse.  

The fastq.gz absolute filepaths may contain environment variables (e.g., `$HOME` or `$PWD`). The following example illustrates a simple fastq manifest file for paired-end read data for two samples.  

```
sample-id,absolute-filepath,direction
# Lines starting with '#' are ignored and can be used to create
# "comments" or even "comment out" entries
sample-1,$PWD/some/filepath/sample1_R1.fastq.gz,forward
sample-2,$PWD/some/filepath/sample2_R1.fastq.gz,forward
sample-1,$PWD/some/filepath/sample1_R2.fastq.gz,reverse
sample-2,$PWD/some/filepath/sample2_R2.fastq.gz,reverse
```

Just like with fastq.gz, the absolute filepaths in the manifest for any fastq files must be accurate. The following example illustrates a simple fastq manifest file for fastq single-end read data for one sample.  
```
sample-id,absolute-filepath,direction
sample-1,$PWD/some/filepath/sample1_R1.fastq,forward
```

There are four variants of FASTQ data which you must specify to QIIME 2 when importing, and which are defined in the following sections.
  
SingleEndFastqManifestPhred33  
In this variant of the fastq manifest format, the read directions must all either be forward or reverse. This format assumes that the PHRED offset used for the positional quality scores in all of the fastq.gz / fastq files is 33.
 
SingleEndFastqManifestPhred64  
In this variant of the fastq manifest format, the read directions must all either be forward or reverse. This format assumes that the PHRED offset used for the positional quality scores in all of the fastq.gz / fastq files is 64. During import, QIIME 2 will convert the PHRED 64 encoded quality scores to PHRED 33 encoded quality scores. This conversion will be slow, but will only happen one time.

PairedEndFastqManifestPhred33  
In this variant of the fastq manifest format, there must be forward and reverse read fastq.gz / fastq files for each sample id. As a result, each sample id is represented twice in the manifest file: once for its forward reads, and once for its reverse reads. This format assumes that the PHRED offset used for the positional quality scores in all of the fastq.gz / fastq files is 33.

PairedEndFastqManifestPhred64  
In this variant of the fastq manifest format, there must be forward and reverse read fastq.gz / fastq files for each sample id. As a result, each sample id is represented twice in the manifest file: once for its forward reads, and once for its reverse reads. This format assumes that the PHRED offset used for the positional quality scores in all of the fastq.gz / fastq files is 64. During import, QIIME 2 will convert the PHRED 64 encoded quality scores to PHRED 33 encoded quality scores. This conversion will be slow, but will only happen one time.

* Obtain data  
  
> `(qiime2) $ wget \`  
  `-O "se-33.zip" \`  
  `"https://data.qiime2.org/2019.1/tutorials/importing/se-33.zip"`
  
> `(qiime2) $wget \`  
  `-O "se-33-manifest" \`  
  `"https://data.qiime2.org/2019.1/tutorials/importing/se-33-manifest"`  
  
> `(qiime2) $ wget \`  
  `-O "pe-64.zip" \`  
  `"https://data.qiime2.org/2019.1/tutorials/importing/pe-64.zip"`  
  
> `(qiime2) $ wget \`
  `-O "pe-64-manifest" \`
  `"https://data.qiime2.org/2019.1/tutorials/importing/pe-64-manifest"`  
  
> `unzip -q se-33.zip`  
`unzip -q pe-64.zip`  
  
* Importing Data  
  
    * Phred33
> `(qiime2) $ qiime tools import \`  
  `--type 'SampleData[SequencesWithQuality]' \`  
  `--input-path se-33-manifest \`  
  `--output-path single-end-demux.qza \`  
  `--input-format SingleEndFastqManifestPhred33`  
    * Phred64
> `(qiime2) $ qiime tools import \`  
  `--type 'SampleData[PairedEndSequencesWithQuality]' \`  
  `--input-path pe-64-manifest \`  
  `--output-path paired-end-demux.qza \`  
  `--input-format PairedEndFastqManifestPhred64`  

이후의 내용은 추후에 정리하도록 한다.

---

----

## Performing longitudinal and paired sample comparisons with q2-longitudinal  
  
q2-longitudinal의 various features에 대해 시연할 것이다.  
이 plugin은 통계 longitudinal 학습 설계의 visual comparisons과 paired sample을 지원하며,  
이런 것들로 인해 ‘stats’간에 if/how sample이 변경되는지에 대해 결정하게 된다.  
‘stats’는 일반적으로 시간과 environmental gradient과 관련이 있고,  
paired analyses의 경우 서로 다른 시간에서 관찰된 두 시점에서 관찰된 같은 대상으로 구성되어야 한다.  
예를 들어 임상 환자의 대변샘플의 치료받기 전과 치료 후의 샘플을 말하는 것이다.  
  
“States”는 일반적으로 methodological으로 사용할 수 있으며,  
이 sample 두시 점에 대한 같은 sample을 말한다.  
예를 들어, q2-longitudinal는 여러가지 수집방법, 저장 방법, DNA 추출 방법 또는 각 sample의 feature 끼리 비교에 관한 bioinformatic processing step에 비교하는 것이 가능하다.
  
이번 예제에서 사용할 데이터는 ECAM study데이터 이다.  
연구 내용은 2세까지의 영아와 엄마의 microbiota에 대한 longitudinal에 대한 연구이다.  
  
### Obtain data
> `(qiime2) $ mkdir longitudinal-tutorial`  
`(qiime2) $ cd longitudinal-tutorial`  
`(qiime2) $ wget -O "ecam-sample-metadata.tsv" "https://data.qiime2.org/2019.1/tutorials/longitudinal/sample_metadata.tsv"`  
`(qiime2) $ wget   -O "shannon.qza"   "https://data.qiime2.org/2019.1/tutorials/longitudinal/ecam_shannon.qza"`  
`(qiime2) $ wget   -O "unweighted_unifrac_distance_matrix.qza"   "https://data.qiime2.org/2019.1/tutorials/longitudinal/unweighted_unifrac_distance_matrix.qza"`  
  
### Pairwise difference comparisons
  
  Pairwise difference test는 특정 측정 항목의 값이 치료 전후간에 변하는지 여부를 결정한다.  
이번 visualizer는 현재 feature table안의 또는 sample metadata file안의 metadata values에 대한 feature abundance간의 비교를 지원한다.  
Alpha diversity values(e.g., observed sequence variants)와 Beta diversity values(e.g., principal coordinates)가 test를 할 metrics를 비교하는 것에 유용하다.  
그리고 input으로 하나의 metadata가 담겨있는 것을 넣어야 한다.  
> `(qiime2) $ qiime longitudinal pairwise-differences \`  
`--m-metadata-file ecam-sample-metadata.tsv \`  
`--m-metadata-file shannon.qza \`  
`--p-metric shannon \`  
`--p-group-column delivery \`  
`--p-state-column month \`  
`--p-state-1 0 \`  
`--p-state-2 12 \`  
`--p-individual-id-column studyid \`  
`--p-replicate-handling random \`  
`--o-visualization pairwise-differences.qzv `  
  
* pairwise-differences.qzv:  
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Flongitudinal%2Fpairwise-differences.qzv)

pairwise-distances visualizer 또한 두개의 각각 다른 “states”가 바뀐 paired sample에 대해 평가하나,  
metadata column 또는 artifact를 input으로 가져오는 대신에 “pre”와 “post” sample 쌍에 대해 거리를 평가하기 위해 거리 matrix를 사용하고 group-column매개 변수에 지정된대로 서로 다른 paired에 차이가 큰지에 대한 여부를 테스트한다.  
> `(qiime2) $ qiime longitudinal pairwise-distances \`  
`--i-distance-matrix unweighted_unifrac_distance_matrix.qza \`  
`--m-metadata-file ecam-sample-metadata.tsv \`   
`--p-group-column delivery \`  
`--p-state-column month \`  
`--p-state-1 0 \`  
`--p-state-2 12 \`  
`--p-individual-id-column studyid \`  
`--p-replicate-handling random \`  
`--o-visualization pairwise-distances.qzv`  
  
* pairwise-distances.qzv: view 

### Linear mixed effect models
Linear mixed effects(LME) models single response variable과 하나 또는 다수의 independent variable관계에 대해 test한다.  
예를 들어 repeated-measures sampleling 실험에서 dependent sample간의 관찰이 이루어진다.  
이번 그래프는 LME model을 이용하여 회귀 그래프를 그린다.  
추가적으로 반복되는 데이터를 구별하기 위해 individual-id-column매개 변수로 열을 지정해야한다.  
위 매개변수로 table의 ID기능을 수행한다.  
  
쉼표로 구분하여 임의대로 column을 입력할 수 도 있다.  
여기서는 LME를 사용하여 alpha diversity(Shannon diversity index)가 ECAM dataset의 delivery, mode, diet, 그리고 sex 따라 변동 여부를 test한다.  

> `(qiime2) $ qiime longitudinal linear-mixed-effects \`  
`--m-metadata-file ecam-sample-metadata.tsv \`  
`--m-metadata-file shannon.qza \`  
`--p-metric shannon \`  
`--p-group-columns delivery,diet,sex \`  
`--p-state-column month \`  
`--p-individual-id-column studyid \`  
`--o-visualization linear-mixed-effects.qzv \`  
   
* linear-mixed-effects.qzv: 
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Flongitudinal%2Flinear-mixed-effects.qzv)  
  
Visualizer produced는 몇가지 명령어를 담고 있다.  
첫째 input parameters를 보여주는 편리성을 가진 visualization
둘째 학습된 LEM model의 model summary 이 정보는 각 "groups"에 대해 설명한다.
조사해야될 주요 결과는 "model results"와 "model summary"이다.
  
뒤의 내용은 추후에 정리

### Volatility analysis
Volatility visualizer는 dependent variable이 얼마나 continuous한지,  
independent variable가 얼마나 volatile한지 서로 상호 작용하는 line plot을 생성한다.  
Multiple metadata file과 FeatureTable[Relativefrequency] table은 넣은 것으로 interactive visualization에 사용하는 것이 가능하며,  
또한 다른 dependent variables를 y-axis에 대한 plot으로 선택할 수 있다.
  
> `(qiime2) $ qiime longitudinal volatility \`  
`--m-metadata-file ecam-sample-metadata.tsv \`  
`--m-metadata-file shannon.qza \`  
`--p-default-metric shannon \`  
`--p-default-group-column delivery \`  
`--p-state-column month \`  
`--p-individual-id-column studyid \`  
`--o-visualization volatility.qzv`  
  
* volatility.qzv: 
    * [link](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Flongitudinal%2Fvolatility.qzv)  
 
용어 설명:
* cesarean: 자궁
* vagina: 질
  
Volatility대한 자세한 정리는 추후에 정리

### First diferencing to track rate of change  
Time series data를 보는 다른 방법은 시간의 변화에 따른 change 비율이다.  
  
$$
\Delta Y_t = Y_t -Y_{t-1}
$$
  
Y는 metric의 값이고 t가 시간일 때, $\Delta Y_t$값은 $Y_t - Y_{t-1}$으로 계산이 가능하다.  
계산은 고정된 간격으로 시행되므로 $Y_t$나 $Y_{t-1}$이 없을 경우에는 계산되지 않는다.  
> `(qiime2) $ qiime longitudinal first-differences \`  
`--m-metadata-file ecam-sample-metadata.tsv \`   
`--m-metadata-file shannon.qza \`  
`--p-state-column month \`   
`--p-metric shannon \`   
`--p-individual-id-column studyid \`    
`--p-replicate-handling random \`  
`--o-first-differences shannon-first-differences.qza`  
    
출력값은 SampleData[firstDifferences]이며, 시각화 된다.
예를 들어 volatility visualizer하거나 linear-mixed-effects 또는 other method로 분석할 수 있다.
